In [1]:
! pip install torch_geometric
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Sequential as Seq
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GraphNetwork(MessagePassing):
    def __init__(self, in_channels, hidden_dim, msg_dim, out_channels, aggr='add'):
        """
        A simple Graph Neural Network (GNN) using PyTorch Geometric.

        Args:
        - in_channels (int): Number of input node features.
        - hidden_dim (int): Dimension of hidden layers.
        - msg_dim (int): Dimension of message embeddings.
        - out_channels (int): Number of output node features.
        - aggr (str, optional): Aggregation method ('add', 'mean', 'max'). Default is 'add'.
        """
        super(GraphNetwork, self).__init__(aggr=aggr)  # Define aggregation method

        # Message function: transforms concatenated node pairs into edge messages
        self.message_net = Seq(
            Linear(2 * in_channels, hidden_dim),
            ReLU(),
            Linear(hidden_dim, msg_dim)
        )

        # Update function: transforms node features after aggregation
        self.update_net = Seq(
            Linear(msg_dim + in_channels, hidden_dim),
            ReLU(),
            Linear(hidden_dim, out_channels)
        )

    def forward(self, x, edge_index):
        """
        Forward pass through the GNN.

        Args:
        - x (torch.Tensor): Node feature matrix of shape [num_nodes, in_channels].
        - edge_index (torch.Tensor): Edge list in COO format of shape [2, num_edges].

        Returns:
        - Updated node feature matrix of shape [num_nodes, out_channels].
        """
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))  # Add self-loops
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        """
        Defines the message function for edges.

        Args:
        - x_i (torch.Tensor): Node features of receiving nodes (shape: [num_edges, in_channels]).
        - x_j (torch.Tensor): Node features of sending nodes (shape: [num_edges, in_channels]).

        Returns:
        - Message embeddings of shape [num_edges, msg_dim].
        """
        edge_features = torch.cat([x_i, x_j], dim=1)  # Concatenate sender & receiver features
        return self.message_net(edge_features)  # Transform via message network

    def aggregate(self, inputs, index, dim_size=None):
        """
        Aggregation function (sum, mean, or max).

        Args:
        - inputs (torch.Tensor): Messages from neighbors.
        - index (torch.Tensor): Node indices corresponding to messages.

        Returns:
        - Aggregated messages per node.
        """
        out = torch.zeros(dim_size, inputs.size(1), device=inputs.device)  # Initialize output tensor
        out.scatter_add_(0, index.unsqueeze(-1).expand_as(inputs), inputs)  # Aggregate using scatter_add_
        return out

    def update(self, aggr_out, x):
        """
        Node update function.

        Args:
        - aggr_out (torch.Tensor): Aggregated messages for each node.
        - x (torch.Tensor): Original node features.

        Returns:
        - Updated node features.
        """
        combined = torch.cat([x, aggr_out], dim=1)  # Combine original features & aggregated messages
        return self.update_net(combined)  # Apply transformation via update network

# Example Usage
if __name__ == "__main__":
    from torch_geometric.data import Data

    # Define a simple graph with 4 nodes and 5 edges
    edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],  # Source nodes
                               [1, 0, 2, 1, 3, 2]], dtype=torch.long)  # Target nodes

    x = torch.randn(4, 8)  # Random node features (4 nodes, 8 features each)

    # Create a graph data object
    data = Data(x=x, edge_index=edge_index)

    # Define and apply the model
    model = GraphNetwork(in_channels=8, hidden_dim=32, msg_dim=16, out_channels=8)
    updated_x = model(data.x, data.edge_index)

    print("Updated Node Features:")
    print(updated_x)


     |████████████████████████████████| 370 kB 8.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.6 MB 58.7 MB/s            
     |████████████████████████████████| 231 kB 46.3 MB/s            
     |████████████████████████████████| 41 kB 150 kB/s             
Using legacy 'setup.py install' for torch-geometric, since package 'wheel' is not installed.
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
    Running setup.py install for torch-geometric ... done


ModuleNotFoundError: No module named 'torch_sparse'

Running on spring_n=4_dim=2_nt=1000_dt=0.01


Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... one
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=545146 sha256=d7ea2dd577c912673af4bf3dfc7cc23cc723170c3065fafa7c93d866f0cc71ff
  Stored in directory: /home/yi260/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... one
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1119715 sha256=3b93d4735f95f98b0fdee100bb1d5dd93ccbf3f6060e35d77922c61f58b0a2e0
  Stored in directory: /home/yi260/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch-sparse


In [ ]:
# setup model
import pytorch



model = 

